In [1]:
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from scipy.stats import pearsonr
from xgboost import XGBRegressor
from sklearn.base import clone
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import gc

warnings.filterwarnings("ignore")
import os
os.chdir("/Users/deweywang/Desktop/GitHub/FlightRank/")


In [3]:
import os
import joblib
from scripts.config import CFG
class Trainer:
    def __init__(self, model, models_folder):
        """
        model:         sklearn/LightGBM 风格的回归模型实例
        models_folder: (可选) 保存最佳折模型的目录；如果为 None，则不保存
        """
        self.model = model
        self.models_folder = models_folder
        if self.models_folder:
            os.makedirs(self.models_folder, exist_ok=True)
            
    def fit_predict(self, X, y, X_test):
        print(f"Training {self.model.__class__.__name__}\n")
        train_fold_scores = []
        fold_scores = []
        oof_preds = np.zeros(X.shape[0])
        test_preds = np.zeros(X_test.shape[0])

        split = KFold(n_splits=CFG.n_folds, shuffle=False).split(X, y)
        for fold_idx, (train_idx, val_idx) in enumerate(split):

            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            model = clone(self.model)
            print("Start trainning...")
            model.fit(X_train, y_train)
            # 保存本折模型
            fold_folder = os.path.join(self.models_folder, f"fold{fold_idx}")
            os.makedirs(fold_folder, exist_ok=True)
            fold_path = os.path.join(fold_folder, f"{self.model.__class__.__name__}.pkl")
            joblib.dump(model, fold_path)
            print(f"📦 Saved fold {fold_idx} model to {fold_path}")

            y_preds = model.predict(X_val)
            oof_preds[val_idx] = y_preds
            
            y_train_preds = model.predict(X_train)


            temp_test_preds = model.predict(X_test)
            test_preds += temp_test_preds / CFG.n_folds

            fold_score = pearsonr(y_val, y_preds)[0]
            fold_scores.append(fold_score)
            
            train_fold_score = pearsonr(y_train, y_train_preds)[0]
            train_fold_scores.append(train_fold_score)

            print(f"--- Fold {fold_idx} - val_pearson: {fold_score:.6f} - train_pearson: {train_fold_score:.6f}")

            del X_train, y_train, X_val, y_val, y_preds, model, temp_test_preds
            gc.collect()

        overall_score = pearsonr(y, oof_preds)[0]
        mean_score = np.mean(fold_scores)
        std_score = np.std(fold_scores)
        
        print(f"\n------ Overall Score: {overall_score:.6f} - Mean val_pearson: {mean_score:.6f} ± {std_score:.6f} - Mean train_pearson: {np.mean(train_fold_scores):.6f} ± {np.std(train_fold_scores):.6f} ------\n")
        
        return oof_preds, test_preds, fold_scores, train_fold_scores

In [5]:
import os
import gc
import joblib
import numpy as np
from sklearn.base import clone
from sklearn.model_selection import KFold
from scipy.stats import pearsonr

from scripts.config import CFG


def create_time_decay_weights(n: int, decay: float = 0.9) -> np.ndarray:
    pos = np.arange(n)
    normed = pos / (n - 1)
    w = decay ** (1.0 - normed)
    return w * n / w.sum()


def get_time_slices(n_samples: int):
    return [
        {"name": "full_data", "cutoff": 0},
        {"name": "last_75pct", "cutoff": int(0.25 * n_samples)},
        {"name": "last_50pct", "cutoff": int(0.50 * n_samples)}
    ]


class Trainer:
    def __init__(self, model, models_folder):
        self.model = model
        self.models_folder = models_folder
        if self.models_folder:
            os.makedirs(self.models_folder, exist_ok=True)

    def fit_predict(self, X, y, X_test):
        print(f"Training {self.model.__class__.__name__}\n")
        train_fold_scores = []
        fold_scores = []
        oof_preds = np.zeros(X.shape[0])
        test_preds_slices = {s["name"]: np.zeros(X_test.shape[0]) for s in get_time_slices(X.shape[0])}
        oof_preds_slices = {s["name"]: np.zeros(X.shape[0]) for s in get_time_slices(X.shape[0])}

        kf = KFold(n_splits=3, shuffle=False)
        n_total = X.shape[0]
        slices = get_time_slices(n_total)

        for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X, y)):
            print(f"\n🧩 Fold {fold_idx}")
            X_val, y_val = X.iloc[val_idx], y[val_idx]

            for s in slices:
                slice_name, cutoff = s["name"], s["cutoff"]
                effective_train_idx = train_idx[train_idx >= cutoff]
                if len(effective_train_idx) == 0:
                    continue
                subset_X = X.iloc[cutoff:].reset_index(drop=True)
                subset_y = y[cutoff:].reset_index(drop=True)
                rel_idx = train_idx[train_idx >= cutoff] - cutoff

                X_train = subset_X.iloc[rel_idx]
                y_train = subset_y.iloc[rel_idx]
                sw = create_time_decay_weights(len(subset_X))[rel_idx]

                model = clone(self.model)
                print(f"  ⏳ Training slice: {slice_name}, samples: {len(X_train)}")
                model.fit(X_train, y_train, sample_weight=sw)

                fold_folder = os.path.join(self.models_folder, f"fold{fold_idx}_{slice_name}")
                os.makedirs(fold_folder, exist_ok=True)
                fold_path = os.path.join(fold_folder, f"{self.model.__class__.__name__}.pkl")
                joblib.dump(model, fold_path)
                print(f"  📦 Saved {slice_name} model to {fold_path}")

                y_preds = model.predict(X_val)
                valid_mask = val_idx >= cutoff
                if valid_mask.any():
                    masked_idx = val_idx[valid_mask]
                    oof_preds_slices[slice_name][masked_idx] = y_preds[valid_mask]
                if cutoff > 0 and (~valid_mask).any():
                    oof_preds_slices[slice_name][val_idx[~valid_mask]] = oof_preds_slices["full_data"][val_idx[~valid_mask]]
                y_train_preds = model.predict(X_train)
                temp_test_preds = model.predict(X_test)
                test_preds_slices[slice_name] += temp_test_preds / 3

                fold_score = pearsonr(y_val, y_preds)[0]
                train_score = pearsonr(y_train, y_train_preds)[0]
                fold_scores.append(fold_score)
                train_fold_scores.append(train_score)

                print(f"  ✅ {slice_name} - val_pearson: {fold_score:.6f} - train_pearson: {train_score:.6f}")

                del X_train, y_train, y_preds, model, temp_test_preds
                gc.collect()

        # Simple ensemble (mean)
        oof_simple = np.mean(list(oof_preds_slices.values()), axis=0)
        test_simple = np.mean(list(test_preds_slices.values()), axis=0)
        score_simple = pearsonr(y, oof_simple)[0]

        # Weighted ensemble (based on pearson score)
        slice_scores = {s: pearsonr(y, oof_preds_slices[s])[0] for s in oof_preds_slices}
        total_score = sum(slice_scores.values())
        oof_weighted = sum(slice_scores[s] / total_score * oof_preds_slices[s] for s in slice_scores)
        test_weighted = sum(slice_scores[s] / total_score * test_preds_slices[s] for s in slice_scores)
        score_weighted = pearsonr(y, oof_weighted)[0]

        print(f"\nSimple Ensemble Pearson:   {score_simple:.6f}")
        print(f"Weighted Ensemble Pearson: {score_weighted:.6f}")

        overall_score = pearsonr(y, oof_simple)[0]
        mean_score = np.mean(fold_scores)
        std_score = np.std(fold_scores)

        print(f"\n------ Overall Score: {overall_score:.6f} - Mean val_pearson: {mean_score:.6f} ± {std_score:.6f} - Mean train_pearson: {np.mean(train_fold_scores):.6f} ± {np.std(train_fold_scores):.6f} ------\n")

        return oof_weighted, test_weighted, fold_scores, train_fold_scores


In [14]:
import polars as pl

# 讀取
train_filled = pl.read_parquet("train_filled.parquet")

# 要排除的欄位
exclude_cols = [
    'Id', 'ranker_id', 'selected', 'profileId', 'requestDate',
    'miniRules0_percentage', 'miniRules1_percentage',
    'pricingInfo_passengerCount'
]

feature_cols = [c for c in train_filled.columns if c not in exclude_cols]

# 自動找 float64 欄位
float64_cols = [
    c for c in feature_cols
    if train_filled[c].dtype == pl.Float64
]

# 轉成 float32
import psutil

process = psutil.Process()
before_mem = process.memory_info().rss / (1024 * 1024)

# 做某些操作
train_filled = train_filled.with_columns(
    [pl.col(c).cast(pl.Float32) for c in float64_cols]
)

after_mem = process.memory_info().rss / (1024 * 1024)
print(f"記憶體變化: {after_mem - before_mem:.2f} MB")
print(f"Polars估算 DataFrame 大小: {train_filled.estimated_size() / (1024 * 1024):.2f} MB")


記憶體變化: 615.30 MB
Polars估算 DataFrame 大小: 11598.53 MB


In [ ]:

import polars as pl

# 第一步：計算每個ranker_id的出現次數
group_counts = (
    train_filled
    .group_by("ranker_id")
    .agg(pl.count().alias("n_rows"))
)

# 第二步：挑選 >=10 的 ranker_id
valid_ranker_ids = (
    group_counts
    .filter(pl.col("n_rows") >= 1000)
    .select("ranker_id")
)

# 第三步：過濾掉 <10 的
train_filled = (
    train_filled
    .join(valid_ranker_ids, on="ranker_id", how="inner")
)

print(f"刪掉後還有 {train_filled.height} 行資料")


刪掉後還有 6363568 行資料


In [8]:

after_mem = process.memory_info().rss / (1024 * 1024)
print(f"Polars估算 DataFrame 大小: {train_filled.estimated_size() / (1024 * 1024):.2f} MB")


Polars估算 DataFrame 大小: 4067.59 MB


In [ ]:

feature_cols = [col for col in train_filled.columns if col not in exclude_cols]

print(f"Using {len(feature_cols)} features")

X = train_filled.select(feature_cols)
y = train_filled.select('selected')
groups = train_filled.select('ranker_id')

Using 160 features


In [ ]:
import numpy as np
import polars as pl

# 確認這些物件都是Polars DataFrame
# X, y, groups
# 都是 shape [n_rows, n_cols]

# 先把 ranker_id轉list
unique_rankers = groups.select("ranker_id").unique().to_series().to_list()

# 打亂
np.random.seed(42)
np.random.shuffle(unique_rankers)

# 切8:2
n_train = int(0.8 * len(unique_rankers))
train_rankers = set(unique_rankers[:n_train])
val_rankers = set(unique_rankers[n_train:])

# 用 Polars 過濾 train/val
is_train = groups.select(pl.col("ranker_id").is_in(list(train_rankers)).alias("is_train"))

# 先 concat mask
X_with_mask = X.with_columns(is_train)
y_with_mask = y.with_columns(is_train)
groups_with_mask = groups.with_columns(is_train)

# 分割 DataFrame
X_train_df = X_with_mask.filter(pl.col("is_train"))
X_val_df = X_with_mask.filter(~pl.col("is_train"))
y_train_df = y_with_mask.filter(pl.col("is_train"))
y_val_df = y_with_mask.filter(~pl.col("is_train"))
groups_train_df = groups_with_mask.filter(pl.col("is_train"))
groups_val_df = groups_with_mask.filter(~pl.col("is_train"))

# 再轉 numpy (分批)
X_train_np = X_train_df.drop("is_train").to_numpy()
X_val_np = X_val_df.drop("is_train").to_numpy()
y_train_np = y_train_df.drop("is_train").to_numpy().flatten()
y_val_np = y_val_df.drop("is_train").to_numpy().flatten()
groups_train_np = groups_train_df.drop("is_train").to_numpy().flatten()
groups_val_np = groups_val_df.drop("is_train").to_numpy().flatten()

# 最後計算 group sizes
group_sizes_train = (
    groups_train_df.drop("is_train")
    .group_by("ranker_id")
    .agg(pl.len().alias("size"))
    .sort("ranker_id")["size"]
    .to_numpy()
)

group_sizes_val = (
    groups_val_df.drop("is_train")
    .group_by("ranker_id")
    .agg(pl.len().alias("size"))
    .sort("ranker_id")["size"]
    .to_numpy()
)


In [ ]:
import xgboost as xgb

# DMatrix 建立 (不再用 X.columns)
dtrain = xgb.DMatrix(
    X_train_np,
    label=y_train_np,
)
dtrain.set_group(group_sizes_train)

dval = xgb.DMatrix(
    X_val_np,
    label=y_val_np,
)
dval.set_group(group_sizes_val)


In [11]:

# 參數
xgb_params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg@3',
    'max_depth': 10,
    'min_child_weight': 10,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 10.0,
    'learning_rate': 0.05,
    'seed': 42,
    'n_jobs': -1,
}

# 訓練
xgb_model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=500,
    evals=[(dtrain, "train"), (dval, "val")],
    early_stopping_rounds=50,
    verbose_eval=20
)


[0]	train-ndcg@3:0.40046	val-ndcg@3:0.36380
[20]	train-ndcg@3:0.53183	val-ndcg@3:0.40077
[40]	train-ndcg@3:0.56886	val-ndcg@3:0.40368
[60]	train-ndcg@3:0.60412	val-ndcg@3:0.40785
[80]	train-ndcg@3:0.63796	val-ndcg@3:0.41526
[100]	train-ndcg@3:0.66802	val-ndcg@3:0.41978
[120]	train-ndcg@3:0.70568	val-ndcg@3:0.42278
[140]	train-ndcg@3:0.73823	val-ndcg@3:0.42878
[160]	train-ndcg@3:0.76533	val-ndcg@3:0.43217
[180]	train-ndcg@3:0.78308	val-ndcg@3:0.43459
[200]	train-ndcg@3:0.80294	val-ndcg@3:0.43787
[220]	train-ndcg@3:0.81615	val-ndcg@3:0.44211
[240]	train-ndcg@3:0.83163	val-ndcg@3:0.44897
[260]	train-ndcg@3:0.84075	val-ndcg@3:0.44923
[280]	train-ndcg@3:0.85239	val-ndcg@3:0.45233
[300]	train-ndcg@3:0.86052	val-ndcg@3:0.45739
[320]	train-ndcg@3:0.86722	val-ndcg@3:0.45941
[340]	train-ndcg@3:0.87219	val-ndcg@3:0.45965
[360]	train-ndcg@3:0.87792	val-ndcg@3:0.45902
[380]	train-ndcg@3:0.88167	val-ndcg@3:0.45837
[400]	train-ndcg@3:0.88669	val-ndcg@3:0.46120
[420]	train-ndcg@3:0.89189	val-ndcg@3:0.

In [12]:
# 存成二進位模型
xgb_model.save_model("xgb_ranker_model.bin")
print("✅ 已儲存模型到 'xgb_ranker_model.bin'")


✅ 已儲存模型到 'xgb_ranker_model.bin'


In [19]:
# 取得重要性 (index key: f0, f1, ...)
importance_dict = xgb_model.get_score(importance_type="weight")

# 把特徵名稱對應回 X.columns
# 如果X是 Polars DataFrame
feature_names = list(X.columns)

# 重新命名
importance_named = {}
for k, v in importance_dict.items():
    # k = 'f0', 'f1', ...
    idx = int(k[1:])  # 取數字
    real_name = feature_names[idx]
    importance_named[real_name] = v

# 排序
sorted_importance = sorted(importance_named.items(), key=lambda x: x[1], reverse=True)

# 列印
print("⭐ Top 20 Feature Importances (by weight):")
for feat, score in sorted_importance[:20]:
    print(f"{feat}: {score}")


⭐ Top 20 Feature Importances (by weight):
totalPrice: 8578.0
companyID: 6165.0
legs0_segments0_flightNumber: 6092.0
legs1_segments0_flightNumber: 5554.0
days_before_departure: 5454.0
legs1_segments0_seatsAvailable: 3737.0
legs0_segments0_seatsAvailable: 3721.0
legs0_arrivalAt_hour: 3577.0
legs1_arrivalAt_hour: 3280.0
legs0_departureAt_hour: 3152.0
legs1_departureAt_hour: 2996.0
corporateTariffCode: 2835.0
taxes: 2525.0
legs1_departureAt_weekday: 1990.0
legs0_departureAt_weekday: 1817.0
legs0_arrivalAt_weekday: 1588.0
legs1_arrivalAt_weekday: 1501.0
frequentFlyer: 1446.0
legs1_duration: 1376.0
legs0_duration: 1354.0


In [17]:
import pandas as pd

# 先準備特徵名稱
feature_names = list(X.columns)

# 取三種重要性
importance_types = ["weight", "gain", "cover"]
importance_all = {}

for imp_type in importance_types:
    imp_raw = xgb_model.get_score(importance_type=imp_type)
    imp_named = {}
    for k, v in imp_raw.items():
        idx = int(k[1:])
        real_name = feature_names[idx]
        imp_named[real_name] = v
    # 排序
    sorted_imp = sorted(imp_named.items(), key=lambda x: x[1], reverse=True)
    importance_all[imp_type] = sorted_imp

# 把三個榜單放成DataFrame方便比對
df_weight = pd.DataFrame(importance_all["weight"], columns=["feature", "weight_rank"])
df_weight["weight_rank_pos"] = df_weight.index

df_gain = pd.DataFrame(importance_all["gain"], columns=["feature", "gain_rank"])
df_gain["gain_rank_pos"] = df_gain.index

df_cover = pd.DataFrame(importance_all["cover"], columns=["feature", "cover_rank"])
df_cover["cover_rank_pos"] = df_cover.index

# 合併
df_merged = (
    df_weight
    .merge(df_gain, on="feature", how="outer")
    .merge(df_cover, on="feature", how="outer")
)

# 把不存在的rank補大數字
df_merged["weight_rank_pos"] = df_merged["weight_rank_pos"].fillna(9999)
df_merged["gain_rank_pos"] = df_merged["gain_rank_pos"].fillna(9999)
df_merged["cover_rank_pos"] = df_merged["cover_rank_pos"].fillna(9999)

# 計算「三個榜單中最早出現的位置」
df_merged["min_rank"] = df_merged[["weight_rank_pos", "gain_rank_pos", "cover_rank_pos"]].min(axis=1)

# 排序
df_merged_sorted = df_merged.sort_values("min_rank")

# 取前50
top50 = df_merged_sorted.head(50)

# 顯示
print(top50[["feature", "weight_rank", "gain_rank", "cover_rank"]])

# 如果想輸出CSV
top50.to_csv("feature_importance_top50.csv", index=False)
print("✅ 已輸出 feature_importance_top50.csv")


                                        feature  weight_rank  gain_rank  \
76                                   totalPrice       8578.0   4.235002   
22                   legs0_segments0_cabinClass        109.0  42.103062   
1                                     companyID       6165.0   3.913192   
34   legs0_segments1_departureFrom_airport_iata          1.0  41.605595   
26        legs0_segments0_marketingCarrier_code        153.0   3.633880   
62   legs1_segments1_departureFrom_airport_iata          2.0   7.080189   
25                 legs0_segments0_flightNumber       6092.0   4.140559   
0                              both_legs_direct         35.0  21.166515   
60  legs1_segments1_arrivalTo_airport_city_iata          1.0  18.557640   
55                 legs1_segments0_flightNumber       5554.0   4.186655   
16                           legs0_main_carrier         22.0   3.867804   
3                         days_before_departure       5454.0   3.778232   
20    legs0_segments0_bag

In [2]:
import numpy as np
import polars as pl

# 讀取
train_filled = pl.read_parquet("train_filled.parquet")

exclude_cols = [
    'Id', 'ranker_id', 'selected', 'profileId', 'requestDate',
    'miniRules0_percentage', 'miniRules1_percentage',
    'pricingInfo_passengerCount'
]

feature_cols = [c for c in train_filled.columns if c not in exclude_cols]

# 轉 float32
float64_cols = [
    c for c in feature_cols
    if train_filled[c].dtype == pl.Float64
]
train_filled = train_filled.with_columns(
    [pl.col(c).cast(pl.Float32) for c in float64_cols]
)

# 預先計算 group size
group_counts = (
    train_filled
    .group_by("ranker_id")
    .agg(pl.count().alias("n_rows"))
)

# 選 >=1000 為 train
train_ranker_ids = (
    group_counts
    .filter(pl.col("n_rows") >= 1000)
    .select("ranker_id")
    .to_series()
    .to_list()
)

# 選 >10 為 valid pool
valid_ranker_ids = (
    group_counts
    .filter(pl.col("n_rows") > 10)
    .select("ranker_id")
    .to_series()
    .to_list()
)

# Shuffle valid_rankers
np.random.seed(42)
np.random.shuffle(valid_ranker_ids)

# 切分 valid 80:20
n_valid_train = int(len(valid_ranker_ids) * 0.8)
valid_train_rankers = set(valid_ranker_ids[:n_valid_train])
valid_val_rankers = set(valid_ranker_ids[n_valid_train:])

# 標記每一行屬於 train/val
train_mask = train_filled.select(
    pl.col("ranker_id").is_in(train_ranker_ids).alias("is_train")
)
val_mask = train_filled.select(
    pl.col("ranker_id").is_in(list(valid_val_rankers)).alias("is_val")
)

# X,y,groups
X = train_filled.select(feature_cols)
y = train_filled.select('selected')
groups = train_filled.select('ranker_id')

# concat mask
# 建立 mask
train_mask_df = train_filled.select(
    pl.col("ranker_id").is_in(train_ranker_ids).alias("is_train")
)
val_mask_df = train_filled.select(
    pl.col("ranker_id").is_in(list(valid_val_rankers)).alias("is_val")
)

# 取 Series
train_mask = train_mask_df["is_train"]
val_mask = val_mask_df["is_val"]

# 加進 DataFrame
X_with_mask = X.with_columns([train_mask, val_mask])
y_with_mask = y.with_columns([train_mask, val_mask])
groups_with_mask = groups.with_columns([train_mask, val_mask])


# 分出train
X_train_df = X_with_mask.filter(pl.col("is_train"))
y_train_df = y_with_mask.filter(pl.col("is_train"))
groups_train_df = groups_with_mask.filter(pl.col("is_train"))

# 分出val
X_val_df = X_with_mask.filter(pl.col("is_val"))
y_val_df = y_with_mask.filter(pl.col("is_val"))
groups_val_df = groups_with_mask.filter(pl.col("is_val"))

# 轉numpy
X_train_np = X_train_df.drop(["is_train", "is_val"]).to_numpy()
y_train_np = y_train_df.drop(["is_train", "is_val"]).to_numpy().flatten()
groups_train_np = groups_train_df.drop(["is_train", "is_val"]).to_numpy().flatten()

X_val_np = X_val_df.drop(["is_train", "is_val"]).to_numpy()
y_val_np = y_val_df.drop(["is_train", "is_val"]).to_numpy().flatten()
groups_val_np = groups_val_df.drop(["is_train", "is_val"]).to_numpy().flatten()

# 計算 group sizes
group_sizes_train = (
    pl.DataFrame({"ranker_id": groups_train_np})
    .group_by("ranker_id")
    .agg(pl.len().alias("size"))
    .sort("ranker_id")["size"]
    .to_numpy()
)

group_sizes_val = (
    pl.DataFrame({"ranker_id": groups_val_np})
    .group_by("ranker_id")
    .agg(pl.len().alias("size"))
    .sort("ranker_id")["size"]
    .to_numpy()
)

print("✅ 資料切分完成")
print(f"Train: {X_train_np.shape[0]} rows, {len(np.unique(groups_train_np))} groups")
print(f"Val: {X_val_np.shape[0]} rows, {len(np.unique(groups_val_np))} groups")


✅ 資料切分完成
Train: 6363568 rows, 3160 groups
Val: 3580308 rows, 17930 groups


In [ ]:
import xgboost as xgb

# DMatrix 建立 (不再用 X.columns)
dtrain = xgb.DMatrix(
    X_train_np,
    label=y_train_np,
)
dtrain.set_group(group_sizes_train)

dval = xgb.DMatrix(
    X_val_np,
    label=y_val_np,
)
dval.set_group(group_sizes_val)


: 

In [ ]:

# 參數
xgb_params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg@3',
    'max_depth': 10,
    'min_child_weight': 10,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 10.0,
    'learning_rate': 0.05,
    'seed': 42,
    'n_jobs': -1,
}

# 訓練
xgb_model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=500,
    evals=[(dtrain, "train"), (dval, "val")],
    early_stopping_rounds=50,
    verbose_eval=20
)


[0]	train-ndcg@3:0.38412	val-ndcg@3:0.67881
[20]	train-ndcg@3:0.51979	val-ndcg@3:0.69860
[40]	train-ndcg@3:0.54846	val-ndcg@3:0.70078
[60]	train-ndcg@3:0.58042	val-ndcg@3:0.70415


In [ ]:
import xgboost as xgb
model = xgb.Booster()
model.load_model("xgb_ranker_model.bin")


In [7]:
val_preds = model.predict(dval)


In [1]:
from scripts.hitrate import compute_hitrate_at_3
# 預測
# 計算 HitRate
compute_hitrate_at_3(groups_val_np, y_val_np, val_preds)


NameError: name 'groups_val_np' is not defined

In [ ]:
import numpy as np
import polars as pl

# 讀取
train_filled = pl.read_parquet("train_filled.parquet")

# 排除欄
exclude_cols = [
    'Id', 'ranker_id', 'selected', 'profileId', 'requestDate',
    'miniRules0_percentage', 'miniRules1_percentage',
    'pricingInfo_passengerCount'
]
feature_cols = [c for c in train_filled.columns if c not in exclude_cols]

# 轉 float32
float64_cols = [c for c in feature_cols if train_filled[c].dtype == pl.Float64]
train_filled = train_filled.with_columns(
    [pl.col(c).cast(pl.Float32) for c in float64_cols]
)

# 計算group size
group_counts = (
    train_filled
    .group_by("ranker_id")
    .agg(pl.count().alias("n_rows"))
)

# 標註分群
group_counts = group_counts.with_columns([
    pl.when(pl.col("n_rows") <= 19).then("small")
     .when((pl.col("n_rows") >=20) & (pl.col("n_rows") <=154)).then("medium")
     .otherwise("large")
     .alias("group_category")
])

# 把 group_category join 回去
train_filled = train_filled.join(group_counts, on="ranker_id", how="left")

# 確認有標籤
print(train_filled.select("group_category").unique())

# 依分群切分
data_small = train_filled.filter(pl.col("group_category") == "small")
data_medium = train_filled.filter(pl.col("group_category") == "medium")
data_large = train_filled.filter(pl.col("group_category") == "large")

print(f"Small: {data_small.height} rows")
print(f"Medium: {data_medium.height} rows")
print(f"Large: {data_large.height} rows")
